<a href="https://colab.research.google.com/github/Metallicode/Math/blob/main/Boosting_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Boosting Methods

*Boosting works by combining multiple weak learners to create a strong learner. **A weak learner** is a model that performs only slightly better than random guessing, while **a strong learner** is a model that has high accuracy on the task.*

* Sequential Learning: Unlike Random Forests, which train each tree independently, boosting methods train learners sequentially. Each subsequent model is built to correct the mistakes of its predecessor.

* Weighted Training Data: After each round, instances that were misclassified by the current weak learner are given higher weights, making it more likely for subsequent learners to focus on them. Similarly, correctly classified instances are given lower weights.

* Aggregation: The final prediction is typically a weighted combination of the predictions of all the weak learners. For regression tasks, this is often a weighted sum. For classification tasks, it can be a weighted majority vote.

* Regularization via Shrinkage: Boosting algorithms often introduce a learning rate (or shrinkage) parameter. This slows down the learning process by shrinking the contribution of each weak learner, which often leads to better performance and less overfitting.

* Strengths: Often provides higher accuracy than other algorithms, especially on tabular data.
Effective with imbalanced datasets by focusing more on the underrepresented class.

* Challenges: More prone to overfitting, especially with noisy data or when the dataset is small.
Typically slower to train than bagging methods like Random Forests due to the sequential nature.
Loses some interpretability when compared to a single decision tree.

* Hyperparameters: Boosting methods come with their set of hyperparameters (like the depth of the trees, learning rate, and number of trees) that need to be tuned for optimal performance.

*In essence, boosting is a versatile and powerful method in the realm of machine learning, particularly for structured/tabular data. It leverages the idea that "the whole is greater than the sum of its parts," turning a series of weak models into a highly accurate combined model.*

##Examples of Boosting Algorithms:

> **AdaBoost (Adaptive Boosting)**: One of the first successful boosting algorithms. It adjusts the weights of misclassified instances and combines learners through weighted majority voting.

> **Gradient Boosting Machines (GBM)**: Builds trees sequentially, where each tree tries to correct the residuals (the differences between the predicted and true values) of the previous one. It generalizes the boosting procedure to optimize arbitrary loss functions.

> **XGBoost, LightGBM, CatBoost**: Modern and efficient implementations of gradient boosting that offer faster computation and additional features.

##Soft vs. Hard Voting


When using ensemble methods, especially in the context of classification, there are two main ways to aggregate predictions from multiple classifiers: soft voting and hard voting.

**Hard Voting:**
> Each classifier in the ensemble "votes" for a class label.
The class label that gets the majority of the votes is chosen as the final predicted class.
Example: Suppose you have three classifiers and two possible classes (A and B). If the classifiers predict A, A, and B, respectively, then the ensemble prediction using hard voting would be A, as it's the majority class predicted.

**Soft Voting:**

> Each classifier provides a probability for each class label.
The probabilities for each class are averaged across all classifiers.
The class with the highest average probability is chosen as the final predicted class.
Example: Using the same scenario with two possible classes (A and B), let's say the classifiers provide the following probabilities for class A: 0.7, 0.2, and 0.8. The average probability for class A would be (0.7 + 0.2 + 0.8) / 3 = 0.567. If the average for class B is lower than this, then the ensemble prediction using soft voting would be A.

**Which to Use?:**

**Soft voting** is generally preferred over hard voting, especially when the classifiers are well-calibrated. The reason is that soft voting takes into account the confidence levels of individual classifiers, leading to potentially more accurate ensemble predictions.

**Hard voting** can be more robust in cases where the probability outputs from classifiers aren't reliable or well-calibrated.

*In conclusion, while hard voting is a simple majority rule approach, soft voting provides a more nuanced way to aggregate predictions by considering the confidence of individual models.

##AdaBoost

short for "Adaptive Boosting".

one of the most popular ensemble methods that employs the boosting technique.


**AdaBoost in Simple Terms:**

> Imagine you're trying to differentiate between cats and dogs, but you're not very good at it. So, you ask a friend (a "weak learner") to help. Your friend makes some mistakes but gets some right.

> Instead of getting upset at the mistakes, you focus more on the pictures your friend got wrong and ask another friend to try and classify those.

> This second friend also makes mistakes, but again, instead of being upset, you focus on the pictures that are still wrong.

> You repeat this process with many friends. Each friend tries to correct the mistakes of the previous one.

> In the end, you combine all your friends' decisions. Each friend gets a say, but friends who were more confident and accurate have a louder voice in the final decision.

*This process is basically AdaBoost! Each friend is a "weak learner" (often a simple decision tree). Mistakes made by previous learners are given more emphasis by subsequent learners. In the end, you combine all their decisions for a final, strong decision.*

##AdaBoost Formula

1. Weight Initialization: Each data point is given an equal weight initially.

2. For Each Learner:

> * Fit the learner on the dataset considering the weights.
* Calculate the error of the learner.
* Compute the learner's weight in the final decision.
Learners with lower error have more weight (a louder voice).
* Increase the weights of the misclassified points, so the next learner pays more attention to them.

3. Final Output: Combine the decisions of all learners. Each learner's decision is weighted by its accuracy.

###Error of a Learner

```
learner_error = Sum_of_weights_of_misclassified_points / Total_Sum_of_weights

```


###Weight of a learner (alpha)

```
alpha = (1/2)*natural_log(1-error/error)
```

###Updating Weights of Data Points

**For correctly classified points:**

```
new_w = old_w * np.e**-alpha
```

**For misclassified points:**
```
new_w = old_w * np.e**alpha
```

###Final Prediction

Mathematically, if h_t(x) is the prediction of the t'th weak learner for an input x and α_t is the weight of that learner, then the aggregated score for x is given by

```
(Where T is the total number of weak learners)

H(x) = sum(α_t*h_t(x) for t in T)

Output = Sign(H(x))

```

##Simple implementation of AdaBoost

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

##np.dot

In [ ]:
#vector dot product x[0]*y[0] + x[1]*y[1]  + x[2]*y[2] .....

n_samples = 10
sample_weights = np.ones(n_samples) / n_samples
#print(sample_weights)
dummy_preds = [True, False, True, True, True, True, False, False, True, True]
np.dot(sample_weights, dummy_preds)

##AdaBoost

In [26]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np

# Toy dataset: 10 data points and 2 features
X = np.array([[1, 2], [2, 3], [3, 1], [4, 2], [5, 2], [6, 1], [7, 2], [8, 3], [9, 1], [10, 2]])
y = np.array([-1, -1, -1, -1, 1, 1, -1, 1, 1, 1])

# Parameters
n_samples, n_features = X.shape
n_learners = 10
learners = []
learner_weights = []
sample_weights = np.ones(n_samples) / n_samples

for i in range(n_learners):
    # Fit a weak learner
    learner = DecisionTreeClassifier(max_depth=1)
    learner.fit(X, y, sample_weight=sample_weights)

    # Predict using the weak learner
    predictions = learner.predict(X)

    # Compute the error
    misclassified = (predictions != y)
    ##the dot product efficiently computes the numerator of the error formula.
    error = np.dot(sample_weights, misclassified) / np.sum(sample_weights)

    print(f"Iteration {i + 1} - Error: {error:.3f} - Predictions: {predictions}")

    if error == 0:
        print("Perfect classifier found!")
        break
    elif error >= 0.5:
        print("Weak learner is worse than random!")
        continue

    # Compute the weak learner's weight
    alpha = 0.5 * np.log((1 - error) / error)

    # Update data point weights
    sample_weights *= np.exp(-alpha * y * predictions)
    #normalization step
    sample_weights /= np.sum(sample_weights)

    # Save the learner and its weight
    learners.append(learner)
    learner_weights.append(alpha)


# Final prediction
output = np.zeros(n_samples)
for alpha, learner in zip(learner_weights, learners):
    output += alpha * learner.predict(X)
final_predictions = np.sign(output)

print("True labels      :", y)
print("AdaBoost Predictions:", final_predictions)

final_predictions * y


Iteration 1 - Error: 0.100 - Predictions: [-1 -1 -1 -1  1  1  1  1  1  1]
Iteration 2 - Error: 0.111 - Predictions: [-1 -1 -1 -1 -1 -1 -1  1  1  1]
Iteration 3 - Error: 0.219 - Predictions: [ 1  1  1  1  1  1 -1 -1 -1 -1]
Iteration 4 - Error: 0.180 - Predictions: [-1 -1 -1 -1  1  1  1  1  1  1]
Iteration 5 - Error: 0.195 - Predictions: [-1 -1 -1 -1 -1 -1 -1  1  1  1]
Iteration 6 - Error: 0.189 - Predictions: [ 1  1  1  1  1  1 -1 -1 -1 -1]
Iteration 7 - Error: 0.192 - Predictions: [-1 -1 -1 -1  1  1  1  1  1  1]
Iteration 8 - Error: 0.191 - Predictions: [-1 -1 -1 -1 -1 -1 -1  1  1  1]
Iteration 9 - Error: 0.191 - Predictions: [ 1  1  1  1  1  1 -1 -1 -1 -1]
Iteration 10 - Error: 0.191 - Predictions: [-1 -1 -1 -1  1  1  1  1  1  1]
True labels      : [-1 -1 -1 -1  1  1 -1  1  1  1]
AdaBoost Predictions: [-1. -1. -1. -1.  1.  1. -1.  1.  1.  1.]


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

##SAMME

stands for *Stagewise Additive Modeling using a Multi-class Exponential loss function* ,is an extension of the AdaBoost algorithm to handle multi-class classification problems.

The traditional AdaBoost algorithm was designed for binary classification. When it comes to multi-class classification problems, we need to adapt the method, and SAMME is one such adaptation.

Here's a brief overview of SAMME:

1. **Initialization**: As in AdaBoost, we initialize the weights for each data point.

2. **For each iteration**:
    - Train a weak learner on the weighted data.
    - The learner predicts the class labels.
    - Calculate the weighted error rate of the learner.
    - Compute the learner's weight (alpha) based on the error. In SAMME, this weight is also influenced by the number of classes. For two classes, SAMME reduces to the original AdaBoost.
    - Update the data point weights: this is done in a way similar to AdaBoost, but adapted for the multi-class scenario.

3. **Final Model Output**: To make predictions, the output from the individual learners is combined based on their weights. The class with the highest combined weight is predicted.

**SAMME.R**(R stands for "Real") is a variant of SAMME that uses the predicted class probabilities (from the weak learners) instead of just the class predictions, often leading to faster convergence.

In practical terms, if you use libraries like scikit-learn, you'll see options to choose between AdaBoost and SAMME when you want to solve multi-class classification problems using boosting.

##sklearn AdaBoostClassifier

In [37]:
df = pd.read_csv("iris.csv")

In [38]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [39]:
X= df.drop("species", axis=1)
y= df["species"]

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [42]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

###Create and fit AdaBoostClassifier

In [43]:
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200,
    algorithm="SAMME.R",
    learning_rate=0.5)

ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

###Predict

In [44]:
y_preds = ada_clf.predict(X_test)

In [45]:
from sklearn.metrics import classification_report

In [46]:
print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        19
  versicolor       0.88      1.00      0.94        15
   virginica       1.00      0.88      0.93        16

    accuracy                           0.96        50
   macro avg       0.96      0.96      0.96        50
weighted avg       0.96      0.96      0.96        50

